In [1]:
import os
import email
import base64
import numpy as np
import sklearn 

from bs4 import BeautifulSoup as soup

In [2]:
def extract(file_content):
    #d=file_content.split("Subject:")
    #print d[1]
    b = email.message_from_string(file_content)
    if b.is_multipart():
        a=''
        for payload in b.get_payload():
        # if payload.is_multipart(): ...
             a.join(str(payload.get_payload()))
             return a
    else:
        return b.get_payload()
   



In [3]:
def vectorize(body):
    body=body.lower()
    a="hello"
    b="how"
    c="are"
    d="you"
    l=[a,b,c,d]
    a=[0,0,0,0]
    for i,x in enumerate(l):
        if x in body:
            a[i]=1
    return a
    

In [4]:
def create(folder_name):
    
    file_list=os.listdir(folder_name)
    vector=[]
       
    for i in file_list:
        path=os.path.join(folder_name,i)
        f=open(path,'r').read()
        body=extract(f) 
        vector.append(vectorize(body))
    
    return vector
    
    
    
    

In [37]:
spam_vector=np.asarray(create("spam"))
spam_label=np.ones([len(spam_vector),1])

In [38]:
ham_vector=np.asarray(create("easy_ham"))
ham_label=np.zeros([len(ham_vector),1])

type(ham_vector[3])

numpy.ndarray

In [41]:
hard_ham_vector=np.asarray(create("hard_ham"))
hard_ham_label=np.zeros([len(hard_ham_vector),1])

email_data=np.concatenate((spam_vector,ham_vector),axis=0)
email_label=np.concatenate((spam_label,ham_label),axis=0)



In [42]:
from sklearn.model_selection import train_test_split

train_set,test_set=train_test_split(email_data,test_size=0.2,random_state=42)
train_label,test_label=train_test_split(email_label,test_size=0.2,random_state=42)

In [51]:
from sklearn import tree


In [52]:
clf=tree.DecisionTreeClassifier()
clf.fit(train_set,train_label)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(test_label,clf.predict(test_label))

ValueError: Number of features of the model must match the input. Model n_features is 4 and input n_features is 1 